In [48]:
import pandas as pd
import spacy
import json

from distool import SymptomExtractor
from distool.feature_extraction.symptom_collection import SYMPTOM_ID_NAME_KEY, SYMPTOMS_FILE_TOP_KEY


In [49]:
df = pd.read_csv('../data/symptoms_datasets/symptoms_dataset.csv')
df.head()

,Болезнь,Симптомы
0,Полиомелит,температура; головная боль; рвота; слабость в ...
1,Птичий грипп,тяжелая дыхательная недостаточность; вирусная ...
2,Ротавирусная инфекция,тошнота; рвота; диарея; схваткообразные боли в...
3,Сибирская язва (кишечная форма),боли в области желудка; понос; рвота; общая ин...
4,Сибирская язва (легочная форма),озноб; температура; покраснение глаз; чихание;...


In [50]:
df.count()

Болезнь     100
Симптомы     99
dtype: int64

In [51]:
df.dropna(inplace=True)

In [52]:
df.count()

Болезнь     99
Симптомы    99
dtype: int64

In [53]:
df.dtypes

Болезнь     object
Симптомы    object
dtype: object

In [54]:
df['Симптомы'] = df['Симптомы'].map(lambda x: x.split('; '))
df.head()

,Болезнь,Симптомы
0,Полиомелит,"[температура, головная боль, рвота, слабость в..."
1,Птичий грипп,"[тяжелая дыхательная недостаточность, вирусная..."
2,Ротавирусная инфекция,"[тошнота, рвота, диарея, схваткообразные боли ..."
3,Сибирская язва (кишечная форма),"[боли в области желудка, понос, рвота, общая и..."
4,Сибирская язва (легочная форма),"[озноб, температура, покраснение глаз, чихание..."


In [55]:
spacy_model = spacy.load(SymptomExtractor.SPACY_LANG_MODEL_NAME)

In [66]:
from distool.feature_extraction.symptom_collection import SYMPTOM_PATTERNS_KEY, SYMPTOMS_FILE_TOP_KEY

SYMPTOM_LEMMA_PATTERN_KEY = "LEMMA"

symptoms_rules = {}

for cell in df['Симптомы']:
    for symptom in cell:
        preprocessed_symptom = symptom.strip()
        symptoms_doc = spacy_model(preprocessed_symptom)
        symptom_lemmas = [token.lemma_ for token in symptoms_doc]
        symptom_name_id = " ".join(symptom_lemmas)

        if symptom_name_id in symptoms_rules:
            symptom_rule = symptoms_rules[symptom_name_id]
        else:
            symptom_rule = {
                SYMPTOM_ID_NAME_KEY : symptom_name_id,
                SYMPTOM_PATTERNS_KEY : set()
            }

        symptom_patterns_set = set()
        for token in symptoms_doc:
            symptom_pattern_part = (SYMPTOM_LEMMA_PATTERN_KEY, token.lemma_)
            symptom_patterns_set.add(symptom_pattern_part)

        symptom_patterns = [dict(item) for item in symptom_patterns_set]
        symptom_rule[SYMPTOM_PATTERNS_KEY].add(symptom_patterns)
        symptoms_rules[symptom_name_id] = symptom_rule



ValueError: dictionary update sequence element #0 has length 5; 2 is required

In [57]:
symptoms_rules

{'температура': {'id_name': 'температура',
  'patterns': [[{'LEMMA': 'температура'}],
   [{'LEMMA': 'температура'}],
   [{'LEMMA': 'температура'}],
   [{'LEMMA': 'температура'}],
   [{'LEMMA': 'температура'}],
   [{'LEMMA': 'температура'}],
   [{'LEMMA': 'температура'}],
   [{'LEMMA': 'температура'}],
   [{'LEMMA': 'температура'}],
   [{'LEMMA': 'температура'}],
   [{'LEMMA': 'температура'}],
   [{'LEMMA': 'температура'}],
   [{'LEMMA': 'температура'}],
   [{'LEMMA': 'температура'}],
   [{'LEMMA': 'температура'}],
   [{'LEMMA': 'температура'}],
   [{'LEMMA': 'температура'}],
   [{'LEMMA': 'температура'}],
   [{'LEMMA': 'температура'}],
   [{'LEMMA': 'температура'}],
   [{'LEMMA': 'температура'}]]},
 'головной боль': {'id_name': 'головной боль',
  'patterns': [[{'LEMMA': 'головной'}, {'LEMMA': 'боль'}],
   [{'LEMMA': 'головной'}, {'LEMMA': 'боль'}],
   [{'LEMMA': 'головной'}, {'LEMMA': 'боль'}],
   [{'LEMMA': 'головной'}, {'LEMMA': 'боль'}],
   [{'LEMMA': 'головной'}, {'LEMMA': 'боль'}]

In [59]:
symptoms_rules_for_json = {
    SYMPTOMS_FILE_TOP_KEY : symptoms_rules
}

In [61]:
with open('../data/symptoms.json', 'w', encoding='utf8') as file:
    json.dump(symptoms_rules_for_json, file, indent=4, ensure_ascii=False)